In [1]:
%cd /Users/trancatkhanh/Downloads/GAME

/Users/trancatkhanh/Downloads/GAME


In [2]:
import numpy as np
from base.MachiKoro.env import *
#0 Array bias trong mỗi trận
#1 Array bias sau 100k trận 
#2 Lưu số trận 
#3 lưu hệ số đã dùng 
#4 Tỷ lệ thắng khi đấu 10k trận 
#5 Trigger để train 100k trận 
#6 Đếm số lần yếu liên tiếp
#7 Lưu array cho policy yếu 
#8 State của trận đấu
#9 Action của trận đấu
#10 Reward của trận đấu
#11 Số trận lưu state
#12 Trigger policy yếu

perx = [np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,getActionSize())),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((1,2)),np.zeros((1,2)),np.zeros((1,1)),np.array([np.random.rand(getActionSize()) for _ in range(1)]),np.zeros((2000000,getStateSize())),np.zeros((2000000,1)),np.zeros((2000000,1)),np.zeros((1,1)),np.zeros((1,2))]
                         #0                                                      #1                   #2              #3                                                            #4              #5               #6                #7                                                               #8                                   #9               #10                       #11           #12

In [10]:
%cd /Users/trancatkhanh/Downloads/GAME
import numpy as np
from base.MachiKoro.env import *
per_state= [np.array([np.random.rand(getStateSize())-0.55 for _ in range(4)])]
@njit()
def tao_arr(state,per):
    actions = getValidActions(state)
    # Check xem policy có yếu không 
    l = np.where(actions ==1)[0]
    action = np.random.choice(l)
    if per[0][1]@state >1:
        print(action)

    return action,per
win, bias = numba_main_2(tao_arr, 1000,per_state,0)
print(win)

/Users/trancatkhanh/Downloads/GAME
1
53
1
46
1
1
47
53
1
53
1
53
1
47
53
1
53
1
2
1
3
10
29
1
0
53
2
50
1
1
53
1
53
1
2
2
53
1
49
2
1
1
2
2
2
1
51
1
2
1
53
2
216


# Các hàm check tính chất của state


In [18]:
import numpy as np
@njit
def check_min(state, reward):
    just = np.zeros((2000000,2))
    for i in range(len(reward)):
        if reward[i] == -1:
            for j in range(i,len(reward)):
                if reward[j] != -1:
                    reward[i] = reward[j]
                    break
    # replace -1 with 0
    for i in range(len(reward)):
        if reward[i] == -1:
            reward[i] = 0
    # np.mean each array in the state and the corresponding reward
    for i in range(len(reward)):
        just[i][0] = np.mean(state[i])
        just[i][1] = (reward[i][0])
    # count the number of times each value of the state appears and the reward of each value of the state
    count = np.zeros((2000000,3))
    for i in range(len(reward)):
        for j in range(len(count)):
            if just[i][0] == count[j][0]:
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
            if count[j][0] == 0:
                count[j][0] = just[i][0]
                count[j][1] += 1
                count[j][2] += just[i][1]
                break
    # find the index value the lowest rate of reward has the highest number of times appear
    min = 0
    for i in range(len(count)):
        if count[i][1] > count[min][1]:
            min = i
    for i in range(len(count)):
        if count[i][1] == count[min][1] and count[i][2] < count[min][2]:
            min = i

    return count[min][0]
@njit
def check_weaker(min,x):
    if x == min:
        return 1
    else:
        return 0


# Agent Hoàn Chỉnh

In [19]:
@njit()
def p_bias(state,per):
    # Chuyển Mode gồm 3 mode mode 0 - train bias , mode 1 - test bias , mode 2 - train policy yếu ,mode 3 - test policy yếu,mode 4 - lưu state
    if per[4][0][0] > per[5][0][1] and per[4][0][1] == 10000 and per[5][0][0] == 1:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      # Lưu lại kết quả thắng
      per[5][0][1] = 0 
      per[5][0][1] += (per[4][0][0])
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Reset số lần yếu liên tiếp
      per[6][0][0] = 0 
    elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 10000 and per[5][0][0] == 1:
      per[6][0][0] +=1 
      # Chuyển sang train bias
      per[5][0][0] = 0 
      per[2][0][0] = 0
      per[1][0] = np.zeros((1,getActionSize()))
      # Rest số trận thắng và số trận đấu 
      per[4][0][0] = 0
      per[4][0][1] = 0 

    # Bắt đầu mode lưu state
    if per[6][0][0] == 3 and per[11][0][0] < 2000000 :
      # Chuyển sang lưu state 
      per[5][0][0] = 4
      per[2][0][0] = 0
      # Rest số trận thắng và số trận đấu 
      per[4][0][1] = 0 
      per[4][0][0] = 0
      
    # Hết mode lưu state chuyển sang train policy yếu
    if per[5][0][0] == 4 and per[11][0][0] == 2000000:
      per[5][0][0] = 2

    # Bất đầu mode train và test policy yếu hơn
    if per[11][0][0] == 2000000:
      if per[4][0][0] > per[5][0][1] and per[4][0][1] == 10000 and per[5][0][0] == 3:
        per[7][0] = np.copy(per[1][0]/np.max(per[1][0]))
        # Lưu lại kết quả thắng
        per[5][0][1] = 0 
        per[5][0][1] += (per[4][0][0])
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))

      elif per[5][0][1] > per[4][0][0] and per[4][0][1] == 10000 and per[5][0][0] == 3:
        # Chuyển sang train
        per[5][0][0] = 2 
        per[2][0][0] = 0
        per[1][0] = np.zeros((1,getActionSize()))
        # Rest số trận thắng và số trận đấu 
        per[4][0][0] = 0
        per[4][0][1] = 0 
      
    # reset per[1][0] sau 100k trận
    if int(per[2][0][0]) == 100000 and per[11][0][0] < 2000000:
      per[3][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 1
    # reset per[1][0] sau 10k trận
    if int(per[2][0][0]) == 100000 and per[11][0][0] == 2000000:
      per[7][0] = np.copy(per[1][0]/np.max(per[1][0]))
      per[2][0][0] = 0
      per[5][0][0] = 3
      
    if per[11][0][0] == 2000000 and per[12][0][0] == 0:
      per[12][0][0] += check_min(per[8], per[10])

    # Chế độ train bias
    if int(per[2][0][0]) < 100000 and per[5][0][0] == 0  :
      actions = getValidActions(state)
      actions *= per[0][0]
      action = np.argmax(actions)
      if getReward(state) == 1 :
          per[1] += per[0][0]
      if getReward(state) == 0:
          per[0][0] = np.random.rand(getActionSize())

    # Chế độ test bias
    elif int(per[2][0][0]) <10000  and per[5][0][0] == 1 : 
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0])    
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  

    # Chế độ train policy yếu
    elif int(per[2][0][0]) < 100000 and per[5][0][0] == 2 and per[11][0][0] == 2000000 :
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      Check_yeu = check_weaker(per[12][0][0],np.mean(state))
      if Check_yeu == 1:
        per[12][0][1] = Check_yeu
        actions *= per[0][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
      if getReward(state) == 1 and per[12][0][1] == 1:
        per[1] += per[0][0]
        per[12][0][1] = 0 
      if getReward(state) == 0 and per[12][0][1] == 1:
        per[0][0] = np.random.rand(getActionSize())
        per[12][0][1] = 0 

    # chế độ test policy yếu
    elif int(per[2][0][0]) <10000 and  per[5][0][0] == 3 and per[11][0][0] == 2000000:
      actions = getValidActions(state)
      # Check xem policy có yếu không 
      if check_weaker(per[12][0][0],np.mean(state)) == 1:
        actions *= per[7][0]
        action = np.argmax(actions)
      else:
        actions *= per[3][0]
        action = np.argmax(actions)
      if getReward(state) == 1:
        per[4][0][0]+=1
      if getReward(state) != -1:
        per[4][0][1]+=1  
    

    # Báo chuyển sang chế độ lưu state và action
    if per[5][0][0] == 4 and per[11][0][0] < 2000000:
      list_action2 = getValidActions(state)
      action = np.argmax(list_action2*per[3][0]) 


    # Lưu state và action
    if getReward(state) != -2 and per[11][0][0] < 2000000 and per[5][0][0] == 4:
        for i in range(getStateSize()):
          per[8][int(per[11][0][0])][i] += state[i]        
        per[9][int(per[11][0][0])][0] += int(action)
        per[10][int(per[11][0][0])][0] += int(getReward(state))
        per[11][0][0] += 1

    # đếm số ván đấu 
    if getReward(state) != -1 :
        per[2][0][0] += 1
    return action,per

win, bias = numba_main_2(p_bias, 1000000,perx,0)

In [20]:
perx[3]

array([[0.78260739, 0.77263313, 0.84290322, 0.78747521, 0.79836953,
        0.79945718, 0.77887925, 0.80687354, 0.81336792, 0.78174458,
        0.77870634, 0.80140909, 0.79967645, 0.80209171, 0.80642151,
        0.79031479, 0.80838948, 0.78654087, 0.80516529, 0.80352382,
        0.80488817, 0.79059457, 0.80135114, 0.80308896, 0.80344931,
        0.80064709, 0.79891001, 0.80919215, 0.79953191, 0.79487493,
        0.80138617, 0.79120958, 0.80563216, 0.7866194 , 0.93920617,
        1.        , 0.78085638, 0.79306468, 0.87460605, 0.91228571,
        0.69859234, 0.70911985, 0.77468408, 0.71009122, 0.76939859,
        0.73649684, 0.81044628, 0.76627458, 0.75047588, 0.93914642,
        0.93087666, 0.96617743, 0.88862088, 0.25404414]])

In [21]:
perx[7]


array([[0.96689953, 0.98327665, 0.95835588, 0.98234617, 0.96962911,
        0.9786492 , 0.97128162, 0.97066785, 0.96919732, 0.9723349 ,
        0.96446063, 0.97636928, 0.96863587, 0.9695763 , 0.96803911,
        0.97203448, 0.98249978, 0.96807534, 0.96794403, 0.98201835,
        0.99635242, 0.97030255, 0.95850928, 0.96150391, 0.96910151,
        0.97189856, 0.97749874, 0.97010252, 0.96600006, 0.98746942,
        0.98007476, 0.97402587, 0.97341286, 0.96992212, 1.        ,
        0.9777625 , 0.97476848, 0.98059478, 0.97149819, 0.96627531,
        0.97995181, 0.97829973, 0.97274644, 0.96481758, 0.96677826,
        0.98129264, 0.98332444, 0.96186786, 0.979761  , 0.97385083,
        0.97225469, 0.96348016, 0.96120374, 0.96074123]])

In [22]:
perx[12]

array([[0.9537037, 0.       ]])

# Test lại tỷ lệ thắng của level 1 game MachiKoro

**Khi chưa khắc phục policy yếu :24551** 


**Khi đã khắc phục policy yếu :27961**

In [25]:
@njit()
def test(state,per):
    actions = getValidActions(state)
    # Check xem policy có yếu không 
    if check_weaker(per[12][0][0],np.mean(state)) == 1:
      actions *= per[7][0]
      action = np.argmax(actions)
    else:
      actions *= per[3][0]
      action = np.argmax(actions)
    return action,per
win, bias = numba_main_2(test, 10000,perx,1)
print(win)

2601


In [ ]:
@njit